In [1]:
import numpy as np
import scipy.stats as ss
import elfi

In [12]:
def fun(x, batch_size, random_state):
    return np.sum(x, keepdims=True, axis=1)

In [13]:
t1 = elfi.Prior(ss.multivariate_normal, [0, 0])
sim = elfi.Simulator(fun, t1, observed=np.array([[0]]))
d = elfi.Distance('euclidean', sim)

In [14]:
res = elfi.Rejection(sim, batch_size=10)

In [15]:
res.sample(10)

--- Logging error ---
Traceback (most recent call last):
  File "/m/home/home2/27/musaevm1/unix/Documents/RA/elfi/elfi/methods/results.py", line 170, in sample_means_summary
    s += ', '.join(["{}: {:.3g}".format(k, v) for k, v in self.sample_means.items()])
  File "/m/home/home2/27/musaevm1/unix/Documents/RA/elfi/elfi/methods/results.py", line 170, in <listcomp>
    s += ', '.join(["{}: {:.3g}".format(k, v) for k, v in self.sample_means.items()])
TypeError: unsupported format string passed to numpy.ndarray.__format__

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/l/musaevm1/anaconda3/elfi_test/lib/python3.6/logging/__init__.py", line 992, in emit
    msg = self.format(record)
  File "/l/musaevm1/anaconda3/elfi_test/lib/python3.6/logging/__init__.py", line 838, in format
    return fmt.format(record)
  File "/l/musaevm1/anaconda3/elfi_test/lib/python3.6/logging/__init__.py", line 575, in format
    record.message = rec

Method: Rejection
Number of samples: 10
Number of simulations: 1000
Threshold: -3.15
Sample means: 